In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
import ngsolve as ngs
import NgsAMG as amg

### H1

It works, I guess.

In [ ]:
from usrMtgStuff import setupH1Square, testAndSolve

V, a, f, u = setupH1Square(maxh=0.01, nref=0)

c = Preconditioner(a, "NgsAMG.h1_scal", ngs_amg_crs_alg="spw")
a.Assemble()
f.Assemble()

testAndSolve(a.mat, c, u, f.vec)

In [ ]:
# Draw(u);
# for t in Timers():
#     if t["time"] > 0.05:
#       print(f"{t['name']}: {t['time']}")

### Elasticity

In [ ]:
from usrMtgStuff import setupElastBeam, testAndSolve

V, a, f, u = setupElastBeam(N=10, dim=3)

c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis")
a.Assemble()
f.Assemble()

testAndSolve(a.mat, c, u, f.vec)

In [ ]:
Draw(u, deformation=True)

No silver bullet - similar limitations as PETSc, BoomerAMG, etc

In [ ]:
from usrMtgStuff import setupElastBeam, testAndSolve

V, a, f, u = setupElastBeam(N=10, aRatio=1000, dim=2)

c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="spw", ngs_amg_sm_type="bgs", ngs_amg_sp_improve_its=2, ngs_amg_sp_omega=0.8)
# c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis", ngs_amg_on_dofs="select", ngs_amg_subset= "nodalp2", ngs_amg_sm_type="bgs")
# c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis", ngs_amg_lo = False, ngs_amg_dof_ordering="p2Emb", ngs_amg_smooth_after_emb=False, ngs_amg_sp_improve_its=0, ngs_amg_sm_type="bgs")

a.Assemble()
f.Assemble()

testAndSolve(a.mat, c, u, f.vec)

In [ ]:
from usrMtgStuff import setupElastBeam, testAndSolve

V, a, f, u = setupElastBeam(N=10, elStretch=5, dim=3, order=2, nodalP2=True)

# c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis")
# c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis", ngs_amg_on_dofs="select", ngs_amg_subset= "nodalp2", ngs_amg_sm_type="bgs")
c = Preconditioner(a, f"NgsAMG.elast_{V.mesh.dim}d", ngs_amg_crs_alg="mis", ngs_amg_lo = False, ngs_amg_dof_ordering="p2Emb", ngs_amg_smooth_after_emb=False, ngs_amg_sp_improve_its=0, ngs_amg_sm_type="bgs")

a.Assemble()
f.Assemble()

testAndSolve(a.mat, c, u.vec, f.vec)

print(f"\n\n\ty-defo = {u(1.0,0.0)[1]} !")

### HDiv

Coarsens dual graph, maintains "loops" (i.e. vertices/edges in 2d/3d), robust in $\left<\nabla\cdot,\nabla\cdot\right>$ penalty parameter

In [ ]:
def StokesHDGDiscretization(mesh, order, inlet, wall, outlet, hodivfree, proj_jumps, div_div_pen, with_pressure, V, Vhat, nu, diri, elint = ('', '', '', True, True, None, True, None, None, 1, 'wall|inlet', False)):
    V1 = HDiv(mesh, order, diri, hodivfree, False, **('order', 'dirichlet', 'hodivfree', 'RT'))
    V2 = TangentialFacetFESpace(mesh, order, diri, proj_jumps, **('order', 'dirichlet', 'highest_order_dc'))
    V = V1 * V2
    Q = L2(mesh, 0 if hodivfree else order - 1, **('order',))
    (u, uhat) = ()
    (v, vhat) = V.TnT()
    (p, q) = Q.TnT()
    n = specialcf.normal(mesh.dim)
    h = specialcf.mesh_size
    
    def tang(vec = None):
        return vec - vec * n * n

    alpha = 4
    dS = dx(True, **('element_boundary',))
    a = BilinearForm(V, elint, **('eliminate_internal',))
    a += nu * InnerProduct(Grad(u), Grad(v)) * dx
    a += nu * InnerProduct(Grad(u) * n, tang(vhat - v)) * dS
    a += nu * InnerProduct(Grad(v) * n, tang(uhat - u)) * dS
    a += (nu * alpha * order * order / h) * InnerProduct(tang(vhat - v), tang(uhat - u)) * dS
    if div_div_pen is not None:
        aPen = BilinearForm(V, elint, **('eliminate_internal',))
        aPen += nu * InnerProduct(Grad(u), Grad(v)) * dx
        aPen += nu * InnerProduct(Grad(u) * n, tang(vhat - v)) * dS
        aPen += nu * InnerProduct(Grad(v) * n, tang(uhat - u)) * dS
        aPen += (nu * alpha * order * order / h) * InnerProduct(tang(vhat - v), tang(uhat - u)) * dS
        aPen += div_div_pen * nu * InnerProduct(div(u), div(v)) * dx
    else:
        aPen = a
    b = BilinearForm(V, Q, **('trialspace', 'testspace'))
    b += -div(u) * q * dx
    return (V, Q, a, b, aPen)

In [ ]:
from usrMtgStuff import GetValve

dim = 2

valve = GetValve(1, dim, 0.5, 25, 1, 180, 6.4, 7, 5, True, **('N', 'dim', 'R', 'alpha', 'Winlet', 'beta', 'L1', 'L2', 'Linlet', 'closevalve'))
mesh = Mesh(OCCGeometry(valve, dim, **('dim',)).GenerateMesh(0.5, **('maxh',)))
mesh.Curve(3)
diri = 'wall'
outlet = None
uin = None
f_vol = CF((1, 0))

(V, Q, a, b, aPen) = StokesHDGDiscretization(mesh, order=order, diri=diri, nu=nu, div_div_pen=div_div_pen)

amg_cl = NgsAMG.stokes_hdiv_gg_2d if mesh.ngmesh.dim == 2 else NgsAMG.stokes_hdiv_gg_3d

pc_opts = {
    'ngs_amg_max_levels': 40,
    'ngs_amg_max_coarse_size': 1,
    'ngs_amg_clev': 'inv',
    'ngs_amg_log_level': 'extra',
    'ngs_amg_log_level_pc': 'extra',
    'ngs_amg_do_test': True,
    'ngs_amg_mg_cycle': 'V',
    'presVecs': 'P1',
    "ngs_amg_pres_vecs": ["RTZ", "P0", "P1"][0]
}



### Smoothers

Various Gauss-Seidel-type MPI-parallel, multiplicative smoothers that overlap MPI and communication:
* Regular Gauss-Seidel
* Block-Gauss-Seidel
* Hybrid-Block-Gauss-Seidel (quite fast for high-order FEM matrices)

Limitation for Block-smoothers in parallel:
 * Blocks may not cross MPI subdomain boundaries
 * Each DOF is owned by the master rank, it will only be included in blocks on that rank

Some examples:
 * Works: blocks of all cell/face/edge/vertex-DOFs
 * Does not work: face/edge-patch, facet-plus-cells
 * Does not work as expected: element, i.e. cell-plus-face-plus-edge-plus-vertex; Master of each DOF will own it, no update from other ranks!
 